In [1]:
import pandas as pd
import datetime
import time
from time import sleep
from tqdm import tqdm
import os
import sys
import ralib.helper
import ralib.time
import ralib.mkfile

In [2]:
path1 = "/Users/gongchenyin/PycharmProjects/ra/data/201906/最终使用/6月汇总.xlsx"
path2 = "/Users/gongchenyin/PycharmProjects/ra/data/201906/最终使用/员工信息表.xlsx"
path3 = "/Users/gongchenyin/PycharmProjects/ra/data/201906/最终使用/总表.xlsx"

In [3]:
df_perform = pd.read_excel(path1)

In [4]:
df_info = pd.read_excel(path2)

In [5]:
 df_database = pd.read_excel(path3)

In [6]:
df_perform = pd.DataFrame(df_perform, columns=["店名", "工号", "岗位", "姓名", "点号合计", "销售业绩"])
df_perform.fillna({"店名": '-', "岗位": '-', "姓名": '-'}, inplace=True)
df_perform[["店名", "岗位", "姓名"]] = df_perform[["店名", "岗位", "姓名"]].applymap(ralib.helper.rm_space)
df_perform = df_perform[df_perform["岗位"].isin(["SPA师", "理疗师", "专家"])]
df_perform.reset_index(drop=True, inplace=True)
df_perform = df_perform.rename(columns={"店名": "部门"})

In [7]:
df_perform.head()

,部门,工号,岗位,姓名,点号合计,销售业绩
0,华东区域淄博人民东路店,1703024,专家,白社红,122.0,205706.96
1,南方区域合肥铂金店,16122101,SPA师,陈新春,89.0,76673.10
2,南方区域合肥铂金店,16122102,SPA师,王洪伟,38.0,5781.60
3,南方区域合肥铂金店,16122104,SPA师,王珊珊,97.0,83625.75
4,南方区域合肥铂金店,16122108,SPA师,慕薇,34.0,32896.92


In [8]:
df_info = pd.DataFrame(df_info, columns=["二级部门", "部门", "员工号", "岗位名称", "姓名", "入职日期"])
df_info.fillna({"二级部门": '-', "部门": '-', "岗位名称": '-', "姓名": '-', "入职日期": '-'}, inplace=True)
df_info["入职日期"] = df_info["入职日期"].map(ralib.time.to_date)
df_info[["二级部门", "部门", "岗位名称", "姓名", "入职日期"]] = df_info[["二级部门", "部门", "岗位名称", "姓名", "入职日期"]].applymap(ralib.helper.rm_space)
df_info = df_info[df_info["岗位名称"].isin(["SPA师", "理疗师", "专家"])]
df_info.reset_index(drop=True, inplace=True)
df_info = df_info.rename(columns={"二级部门": "区域", "员工号": "工号", "岗位名称": "岗位"})

In [9]:
df_info.head()

,区域,部门,工号,岗位,姓名,入职日期
0,济南区域,济南山东大厦店,120316001,SPA师,张开巧,2018-03-03
1,济南区域,济南山东大厦店,131121001,SPA师,寻鲜鲜,2018-03-01
2,济南区域,济南山东大厦店,140524001,SPA师,邵桂兰,2018-02-24
3,济南区域,济南山东大厦店,140807001,SPA师,董军,2018-03-01
4,济南区域,济南山东大厦店,150507001,SPA师,宋志美,2018-03-04


In [10]:
# pre-process df_database
df_database = pd.DataFrame(df_database, columns=["工号", "姓名", "岗位", "入职日期", "出勤", "点号合计", 
                                                 "销售业绩", "工资年月", "部门", "区域", "最早的入职日期", "最近的入职日期"])
df_database.fillna({"姓名": '-', "岗位": '-', "部门": '-', "区域": '-', "入职日期": '-', 
                    "工资年月": '-', "最早的入职日期": '-', "最近的入职日期": '-'}, inplace=True)
df_database["入职日期"] = df_database["入职日期"].map(ralib.time.to_date)
df_database["工资年月"] = df_database["工资年月"].map(ralib.time.to_date)
df_database["最早的入职日期"] = df_database["最早的入职日期"].map(ralib.time.to_date)
df_database["最近的入职日期"] = df_database["最近的入职日期"].map(ralib.time.to_date)
df_database[["姓名", "岗位", "部门", "区域", "入职日期", "工资年月", "最早的入职日期", "最近的入职日期"]] = df_database[["姓名", "岗位", "部门", 
                                                     "区域", "入职日期", "工资年月", "最早的入职日期", "最近的入职日期"]].applymap(ralib.helper.rm_space)
# 总表中每个员工包含多个observation，可能这个员工从技师变为其他工种，之前的筛选方法就不对了: 应该取反~
not_in_id = df_database[~df_database["岗位"].isin(["SPA师", "理疗师", "专家"])]["工号"]
df_database = df_database[~df_database["岗位"].isin(not_in_id)]
df_database.reset_index(drop=True, inplace=True)
# 补全id为空值的信息

In [11]:
df_database.head()

,工号,姓名,岗位,入职日期,出勤,点号合计,销售业绩,工资年月,部门,区域,最早的入职日期,最近的入职日期
0,151027.0,沈妍,技术总监,-,NaN,2.0,725888.0,2019-06-01,鲁南区域 临清新华路店,济南区域,2015-11-25,2018-05-31
1,151027.0,沈妍,SPA师,2015-11-25,25.0,67.0,36699.0,2018-10-01,济南奥体店,济南区域,2015-11-25,2018-05-31
2,151027.0,沈妍,SPA师,2015-11-25,25.0,54.0,16600.0,2019-02-01,济南奥体店,济南区域,2015-11-25,2018-05-31
3,151027.0,沈妍,SPA师,2015-11-25,27.0,65.0,41370.0,2018-08-01,济南奥体店,济南区域,2015-11-25,2018-05-31
4,151027.0,沈妍,SPA师,2015-11-25,23.0,85.0,133188.0,2018-05-01,济南奥体店,济南区域,2015-11-25,2018-05-31


In [12]:
df_perform = pd.merge(df_perform, df_info, on="工号", how="inner")

In [13]:
df_perform = df_perform.drop(["部门_x", "岗位_x", "姓名_x"], axis=1)

In [14]:
df_perform = df_perform.rename(columns={"姓名_y": "姓名", "部门_y": "部门", "岗位_y": "岗位"})
# exist single worker with more than one rows of information, thus need to modify them
df_perform = df_perform.drop_duplicates()                               # delete duplicated rows
df_perform = df_perform.groupby("工号")[["点号合计", "销售业绩"]].sum()  # sum the worker's performance
df_perform.reset_index(drop=False, inplace=True)
df_perform = pd.merge(df_perform, df_info, on="工号", how="inner")

In [15]:
df_perform.head()

,工号,点号合计,销售业绩,区域,部门,岗位,姓名,入职日期
0,1703024,122.0,205706.96,北京区域,北京华彩店,专家,白社红,2018-09-07
1,16122101,89.0,76673.10,济南区域,济南和谐店,SPA师,陈新春,2016-12-21
2,16122102,38.0,5781.60,济南区域,济南和谐店,SPA师,王洪伟,2016-12-21
3,16122104,97.0,83625.75,济南区域,济南和谐店,SPA师,王珊珊,2016-12-22
4,16122108,34.0,32896.92,济南区域,济南和谐店,SPA师,慕薇,2019-04-01


In [16]:
# update 入职日期
id_and = set(df_perform["工号"]).intersection(set(df_database["工号"]))
for i in range(len(df_perform)):
    id_num = df_perform.loc[i, "工号"]
    if id_num in id_and:
        s = df_database[df_database["工号"] == id_num]["最早的入职日期"].iloc[0]
        if (s != '-') and (s is not pd.NaT):
            df_perform.loc[i, "入职日期"] = s

# generate random numbers for the workers in df_perform
df_perform["随机数"] = ralib.helper.sample_floats(0, 1, len(df_perform))
global region
region = df_perform["区域"].unique()
df_list = []
df_list.append(df_perform)
df_list.append(df_database)

In [17]:
df_perform_copy = df_perform.copy()
df_database_copy = df_database.copy()

In [34]:
df_by_region = df_perform_copy[df_perform_copy["区域"] == "鲁南区域"]
# sort workers by performance, and select the first half of them
df_by_region = df_by_region.sort_values(by="销售业绩", ascending=False)
df_by_region.reset_index(drop=True, inplace=True)
 # ??? corner case
cutoff = len(df_by_region) // 2
if cutoff >= 15:
    df_by_region = df_by_region[:cutoff]
# group the seniors in DataFrame that stores worker performance
df_senior_id = df_by_region[df_by_region["入职日期"].map(ralib.time.is_senior)]
df_senior_id.reset_index(drop=True, inplace=True)